# District Summary

In [1448]:
# Dependencies
import pandas as pd
import os

In [1449]:
# Save path to data set in a varible
schools_file = os.path.join("raw_data", "schools_complete.csv")
students_file = os.path.join("raw_data", "students_complete.csv")

In [1450]:
# Use Pandas to read data
schools_file_pd = pd.read_csv(schools_file)
students_file_pd = pd.read_csv(students_file)

In [1451]:
students_file_pd.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [1452]:
number_of_schools = len(schools_file_pd["School ID"].unique())
total_students = len(students_file_pd["Student ID"].unique())
total_budget = schools_file_pd["budget"].sum()
average_math_score = students_file_pd["math_score"].mean()
average_reading_score = students_file_pd["reading_score"].mean()
number_reading_passing_df = students_file_pd.loc[students_file_pd["reading_score"] >= 70]
number_reading_passing = len(number_reading_passing_df["Student ID"].unique())
percent_passing_reading = number_reading_passing / total_students
number_math_passing_df = students_file_pd.loc[students_file_pd["math_score"] >= 70]
number_math_passing = len(number_math_passing_df["Student ID"].unique())
percent_passing_math = number_math_passing / total_students
overall_passing_rate = (percent_passing_reading + percent_passing_math) / 2

In [1453]:
#Create district summary table
district_summary = pd.DataFrame({
    "Key Metric":["Total Schools", "Total Students", "Total Budget", "Average Math Score", "Average Reading Score", "% of Students Passing Math", "% of Students Passing Reading", "Overall Passing Rate"],
    "Metic Value":[number_of_schools, total_students, total_budget, average_math_score, average_reading_score, percent_passing_math, percent_passing_reading, "N/A"]
})
district_summary

,Key Metric,Metic Value
0,Total Schools,15
1,Total Students,39170
2,Total Budget,24649428
3,Average Math Score,78.9854
4,Average Reading Score,81.8778
5,% of Students Passing Math,0.749809
6,% of Students Passing Reading,0.858055
7,Overall Passing Rate,N/A


# School Summary Section

In [1454]:
#Rename school name column for eventual merge with student table grouped data
renamed_schools_file_pd = schools_file_pd.rename(columns={"name":"school"})

In [1455]:
#Group student table by school with summed reading score and math score columns
school_group_scores = students_file_pd.groupby(["school"], as_index='False')
school_group_scores_total = school_group_scores["reading_score", "math_score"].sum().reset_index('school')
#Rename columns in grouped student table
school_group_scores_total_renamed = school_group_scores_total.rename(columns={"reading_score":"reading_score_all", "math_score":"math_score_all"})

In [1456]:
#Group student reading passing table by school with count of students column
school_group_reading_passing = number_reading_passing_df.groupby(["school"], as_index='False')
school_group_reading_passing_total = school_group_reading_passing["Student ID"].count().reset_index('school')
school_group_reading_passing_total_renamed = school_group_reading_passing_total.rename(columns={"Student ID":"reading_passing_students"})
#Group student math passing table by school with count of students column
school_group_math_passing = number_math_passing_df.groupby(["school"], as_index='False')
school_group_math_passing_total = school_group_math_passing["Student ID"].count().reset_index('school')
school_group_math_passing_total_renamed = school_group_math_passing_total.rename(columns={"Student ID":"math_passing_students"})

In [1457]:
#Merge tables necessary for final table
school_merge_1 = pd.merge(renamed_schools_file_pd, school_group_scores_total_renamed, on="school", how="left")
school_merge_2 = pd.merge(school_merge_1, school_group_reading_passing_total_renamed, on="school", how="left")
school_merge_3 = pd.merge(school_merge_2, school_group_math_passing_total_renamed, on="school", how="left")

In [1458]:
#Add column for budget per student calculation
school_merge_3["Budget Per Student"] = school_merge_3["budget"]/school_merge_3["size"]

#Add column for average math score per student
school_merge_3["Average Math Score"] = school_merge_3["math_score_all"]/school_merge_3["size"]

#Add column for average reading score per student
school_merge_3["Average Reading Score"] = school_merge_3["reading_score_all"]/school_merge_3["size"]

#Add column for percent passing math
school_merge_3["% Passing Math"] = (school_merge_3["math_passing_students"]/school_merge_3["size"])*100

#Add column for percent passing reading
school_merge_3["% Passing Reading"] = (school_merge_3["reading_passing_students"]/school_merge_3["size"])*100

#Add column for overall passing rate
school_merge_3["Overall Pass Rate"] = (school_merge_3["% Passing Math"] + school_merge_3["% Passing Reading"])/2 

In [1459]:
#Remove columns not needed in final table
school_merge_final = school_merge_3.drop(["School ID", "reading_score_all", "math_score_all", "reading_passing_students", "math_passing_students"], axis=1)

In [1460]:
#Rename columns heading for final table
school_merge_final_renamed = school_merge_final.rename(columns={"school":"School Name", "type":"School Type", "size":"Total Students", "budget":"Total School Budget"})

In [1461]:
#Format final table columns
school_merge_final_renamed["Total Students"] = school_merge_final_renamed["Total Students"].map("{:,}".format)
school_merge_final_renamed["Total School Budget"] = school_merge_final_renamed["Total School Budget"].map("${:,.0f}".format)
school_merge_final_renamed["Budget Per Student"] = school_merge_final_renamed["Budget Per Student"].map("${:,.0f}".format)
school_merge_final_renamed["Average Math Score"] = school_merge_final_renamed["Average Math Score"].map("{:,.1f}".format)
school_merge_final_renamed["Average Reading Score"] = school_merge_final_renamed["Average Reading Score"].map("{:,.1f}".format)
school_merge_final_renamed["% Passing Math"] = school_merge_final_renamed["% Passing Math"].map("{:,.1f}%".format)
school_merge_final_renamed["% Passing Reading"] = school_merge_final_renamed["% Passing Reading"].map("{:,.1f}%".format)
school_merge_final_renamed["Overall Pass Rate"] = school_merge_final_renamed["Overall Pass Rate"].map("{:,.1f}%".format)

school_merge_final_renamed

,School Name,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
0,Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,73.5%
1,Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,73.4%
2,Shelton High School,Charter,"1,761","$1,056,600",$600,83.4,83.7,93.9%,95.9%,94.9%
3,Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,73.8%
4,Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,95.3%
5,Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,84.0,93.9%,96.5%,95.2%
6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,95.6%
7,Bailey High School,District,"4,976","$3,124,928",$628,77.0,81.0,66.7%,81.9%,74.3%
8,Holden High School,Charter,427,"$248,087",$581,83.8,83.8,92.5%,96.3%,94.4%
9,Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,95.3%


# Top Performing Schools

In [1462]:
top_five_schools = school_merge_final_renamed.sort_values("Overall Pass Rate", ascending=False)
top_five_schools.head(5)

,School Name,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
6,Cabrera High School,Charter,"1,858","$1,081,356",$582,83.1,84.0,94.1%,97.0%,95.6%
4,Griffin High School,Charter,"1,468","$917,500",$625,83.4,83.8,93.4%,97.1%,95.3%
9,Pena High School,Charter,962,"$585,858",$609,83.8,84.0,94.6%,95.9%,95.3%
14,Thomas High School,Charter,"1,635","$1,043,130",$638,83.4,83.8,93.3%,97.3%,95.3%
5,Wilson High School,Charter,"2,283","$1,319,574",$578,83.3,84.0,93.9%,96.5%,95.2%


# Bottom Performing Schools

In [1463]:
bottom_five_schools = school_merge_final_renamed.sort_values("Overall Pass Rate", ascending=True)
bottom_five_schools.head(5)

,School Name,School Type,Total Students,Total School Budget,Budget Per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Pass Rate
11,Rodriguez High School,District,"3,999","$2,547,363",$637,76.8,80.7,66.4%,80.2%,73.3%
1,Figueroa High School,District,"2,949","$1,884,411",$639,76.7,81.2,66.0%,80.7%,73.4%
0,Huang High School,District,"2,917","$1,910,635",$655,76.6,81.2,65.7%,81.3%,73.5%
12,Johnson High School,District,"4,761","$3,094,650",$650,77.1,81.0,66.1%,81.2%,73.6%
3,Hernandez High School,District,"4,635","$3,022,020",$652,77.3,80.9,66.8%,80.9%,73.8%


# Math Scores By Grade

In [1464]:
students_file_pd["Student Count"] = 1

In [1465]:
students_grouped = students_file_pd.groupby(["school", "grade"], as_index='False')
students_grouped_summary = students_grouped["math_score", "reading_score", "Student Count"].sum().reset_index('grade').reset_index('school')

In [1466]:
#Add column for average math score
students_grouped_summary["Average Math Score"] = students_grouped_summary["math_score"]/students_grouped_summary["Student Count"]
#Add column for average reading score
students_grouped_summary["Average Reading Score"] = students_grouped_summary["reading_score"]/students_grouped_summary["Student Count"]

In [1467]:
#Create dataframes for grade columns
ninth_grade = students_grouped_summary.loc[students_grouped_summary["grade"] == "9th",:]
ninth_grade = ninth_grade.drop(["grade", "math_score", "reading_score", "Student Count"], axis=1)
ninth_grade = ninth_grade.rename(columns={"Average Math Score":"Average Math Score 9", "Average Reading Score":"Average Reading Score 9"})
tenth_grade = students_grouped_summary.loc[students_grouped_summary["grade"] == "10th",:]
tenth_grade = tenth_grade.drop(["grade", "math_score", "reading_score", "Student Count"], axis=1)
tenth_grade = tenth_grade.rename(columns={"Average Math Score":"Average Math Score 10", "Average Reading Score":"Average Reading Score 10"})
eleventh_grade = students_grouped_summary.loc[students_grouped_summary["grade"] == "11th",:]
eleventh_grade = eleventh_grade.drop(["grade", "math_score", "reading_score", "Student Count"], axis=1)
eleventh_grade = eleventh_grade.rename(columns={"Average Math Score":"Average Math Score 11", "Average Reading Score":"Average Reading Score 11"})
twelfth_grade = students_grouped_summary.loc[students_grouped_summary["grade"] == "12th",:]
twelfth_grade = twelfth_grade.drop(["grade", "math_score", "reading_score", "Student Count"], axis=1)
twelfth_grade = twelfth_grade.rename(columns={"Average Math Score":"Average Math Score 12", "Average Reading Score":"Average Reading Score 12"})

In [1468]:
#Merge grade dataframes
grade_merge1 = pd.merge(ninth_grade, tenth_grade, on="school", how="left")
grade_merge2 = pd.merge(grade_merge1, eleventh_grade, on="school", how="left")
grade_merge3 = pd.merge(grade_merge2, twelfth_grade, on="school", how="left")

In [1469]:
#Create math grade summary table
math_grade_table = grade_merge3.drop(["Average Reading Score 9", "Average Reading Score 10", "Average Reading Score 11", "Average Reading Score 12"], axis=1)
math_grade_table = math_grade_table.rename(columns={"Average Math Score 9":"9th", "Average Math Score 10":"10th", "Average Math Score 11":"11th", "Average Math Score 12":"12th"})
math_grade_table

,school,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Scores By Grade

In [1470]:
#Create reading grade summary table
reading_grade_table = grade_merge3.drop(["Average Math Score 9", "Average Math Score 10", "Average Math Score 11", "Average Math Score 12"], axis=1)
reading_grade_table = reading_grade_table.rename(columns={"Average Reading Score 9":"9th", "Average Reading Score 10":"10th", "Average Reading Score 11":"11th", "Average Reading Score 12":"12th"})
reading_grade_table

,school,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


# Average Spending

In [1471]:
#Create bins and group names
bins = [0, 591, 629, 641, 1000]
group_names = ["4th Quartile - less than $591", "3rd Quartile - between $591 and $629", "2nd Quartile - between $629 & $641", "1st Quartile - greater than $641"]

In [1472]:
#Cut the names for the bins and add to dataframe
school_merge_3["Spending Per Student Quartile"] = pd.cut(school_merge_3["Budget Per Student"], bins, labels=group_names)

In [1473]:
#Add school counter into dataframe
school_merge_3["School Count"] = 1

In [1474]:
#Group the dataframe by spending per student quartiles
spending_groups = school_merge_3.groupby(["Spending Per Student Quartile"], as_index='False')
spending_groups_summary = spending_groups["School Count","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"].sum().reset_index('Spending Per Student Quartile')

In [1475]:
#Calculate measures at quartile level
spending_groups_summary["Quartile Average Math Score"] = spending_groups_summary["Average Math Score"]/spending_groups_summary["School Count"]
spending_groups_summary["Quartile Average Reading Score"] = spending_groups_summary["Average Reading Score"]/spending_groups_summary["School Count"]
spending_groups_summary["Quartile % Passing Math"] = spending_groups_summary["% Passing Math"]/spending_groups_summary["School Count"]
spending_groups_summary["Quartile % Passing Reading"] = spending_groups_summary["% Passing Reading"]/spending_groups_summary["School Count"]
spending_groups_summary["Quartile Overall Pass Rate"] = spending_groups_summary["Overall Pass Rate"]/spending_groups_summary["School Count"]

In [1476]:
#Remvoe columsn not needed in final table
spending_groups_final = spending_groups_summary.drop(["School Count", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"], axis=1)
spending_groups_final

,Spending Per Student Quartile,Quartile Average Math Score,Quartile Average Reading Score,Quartile % Passing Math,Quartile % Passing Reading,Quartile Overall Pass Rate
0,4th Quartile - less than $591,83.455399,83.933814,93.460096,96.610877,95.035486
1,3rd Quartile - between $591 and $629,81.899826,83.155286,87.133538,92.718205,89.925871
2,2nd Quartile - between $629 & $641,78.990942,81.917212,75.209078,86.089386,80.649232
3,1st Quartile - greater than $641,77.023555,80.957446,66.701010,80.675217,73.688113


# School Size

In [1477]:
#Create bins and group names
bins = [0, 2000, 2800, 10000]
group_names = ["Samll less than 2,000", "Medium between 2,000 and 2,800", "Large greater than 2,800"]

In [1478]:
#Cut the names for the bins and add to dataframe
school_merge_3["School Size"] = pd.cut(school_merge_3["size"], bins, labels=group_names)

In [1479]:
#Group the dataframe by school size
size_groups = school_merge_3.groupby(["School Size"], as_index='False')
size_groups_summary = size_groups["School Count","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"].sum().reset_index('School Size')

In [1480]:
#Calculate measures at size group level
size_groups_summary["School Size Average Math Score"] = size_groups_summary["Average Math Score"]/size_groups_summary["School Count"]
size_groups_summary["School Size Average Reading Score"] = size_groups_summary["Average Reading Score"]/size_groups_summary["School Count"]
size_groups_summary["School Size % Passing Math"] = size_groups_summary["% Passing Math"]/size_groups_summary["School Count"]
size_groups_summary["School Size % Passing Reading"] = size_groups_summary["% Passing Reading"]/size_groups_summary["School Count"]
size_groups_summary["School Size Overall Pass Rate"] = size_groups_summary["Overall Pass Rate"]/size_groups_summary["School Count"]

In [1481]:
#Remvoe columsn not needed in final table
size_groups_final = size_groups_summary.drop(["School Count", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"], axis=1)
size_groups_final

,School Size,School Size Average Math Score,School Size Average Reading Score,School Size % Passing Math,School Size % Passing Reading,School Size Overall Pass Rate
0,"Samll less than 2,000",83.502373,83.883125,93.585560,96.593182,95.089371
1,"Medium between 2,000 and 2,800",80.188396,82.367873,81.088660,87.919328,84.503994
2,"Large greater than 2,800",76.932423,81.003366,66.254928,81.049070,73.651999


# School Type

In [1482]:
#Group the dataframe by school type
type_groups = school_merge_3.groupby(["type"], as_index='False')
type_groups_summary = type_groups["School Count","Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"].sum().reset_index('type')

In [1483]:
#Calculate measures at type group level
type_groups_summary["School Type Average Math Score"] = type_groups_summary["Average Math Score"]/type_groups_summary["School Count"]
type_groups_summary["School Type Average Reading Score"] = type_groups_summary["Average Reading Score"]/type_groups_summary["School Count"]
type_groups_summary["School Type % Passing Math"] = type_groups_summary["% Passing Math"]/type_groups_summary["School Count"]
type_groups_summary["School Type % Passing Reading"] = type_groups_summary["% Passing Reading"]/type_groups_summary["School Count"]
type_groups_summary["School Type Overall Pass Rate"] = type_groups_summary["Overall Pass Rate"]/type_groups_summary["School Count"]

In [1484]:
#Remvoe columsn not needed in final table
type_groups_final = type_groups_summary.drop(["School Count", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "Overall Pass Rate"], axis=1)
type_groups_final

,type,School Type Average Math Score,School Type Average Reading Score,School Type % Passing Math,School Type % Passing Reading,School Type Overall Pass Rate
0,Charter,83.473852,83.896421,93.620830,96.586489,95.103660
1,District,76.956733,80.966636,66.548453,80.799062,73.673757
